# tests towards "tableau" version of phreeqc
# does work. 

I want to be able to generate a PHREEQC input file from a tableau
so that my solver can call phreeqc as a possible engine

this file is just trying to test that.  make minimal phreeqc database file and "turn off" activity corrections using large "a" and zero "b" parameter in the correction (gamma values)

fix pH at 4 the normal way with a fixed ph phase

In [1]:
% setup environment
% clear data and number format
clear; format short e
% set path to equilibrium solver
txt=pwd; n=length(txt); rootpath=txt;
n=strfind(rootpath,"USERS"); rootpath=txt(1:n-2);
PHREEQCpath=[rootpath,'/runPHREEQC'];
addpath(PHREEQCpath);
JEQUILIBpath=[rootpath,'/JEQUILIB'];
addpath(JEQUILIBpath);
% turn off warnings
warning off
%graphics_toolkit ("notebook"); 
graphics_toolkit ("plotly"); 
%graphics_toolkit("gnuplot")
%delete existing phreeqc database file
system("rm MINIMALH2O.dat");
%create empty database file
system("touch MINIMALH2O.dat");
system("rm runfile.txt");
system("touch runfile.txt");

In [2]:
MINIMALDATABASE=[...
{'SOLUTION_MASTER_SPECIES\n'}
{'  \n'}
{'#element species        alk     gfw_formula     element_gfw\n'}
{'  \n'}
{'H        H+             -1.     H               1.008\n'}
{'H(0)     H2             0.0     H\n'}
{'H(1)     H+             -1.     0.0\n'}
{'E        e-             0.0     0.0             0.0\n'}
{'O        H2O            0.0     O               16.00\n'}
{'O(0)     O2             0.0     O\n'}
{'O(-2)    H2O            0.0     0.0\n'}
{'Cl       Cl-            0.0     Cl              35.4527\n'}
{'Cl(-1)   Cl-            0       Cl\n'}
{'Na	   Na+ 	          0.0     Na		22.9898\n'}
{'\n'}
{'SOLUTION_SPECIES\n'}
{'# use -gamma 1e10 0.0 to counteract automatic activity correction\n'}
{'  \n'}
{'H+ = H+\n'}
{'      log_k           0.000\n'} 
{'      -gamma          1e10     0.0\n'}
{'  \n'}
{'e- = e-\n'}
{'        log_k           0.000\n'}
{'  \n'}
{'H2O = H2O\n'}
{'        log_k           0.000\n'}
{'  \n'}
{'H2O = OH- + H+\n'}
{'        log_k           -14.000\n'} 
{'        -gamma          1e10     0.0\n'}
{'  \n'}
{'2 H2O = O2 + 4 H+ + 4 e-\n'}
{'        log_k           -86.08\n'}
{'        -gamma          1e10     0.0\n'}
{'  \n'}
{'2 H+ + 2 e- = H2\n'}
{'        log_k           -3.15\n'}
{'        -gamma          1e10     0.0\n'}
{'Cl- = Cl-\n'}
{'	log_k	0.0\n'}
{'	-gamma	1e10 	0.0\n'}
{'Na+ = Na+\n'}
{'	log_k	0.0\n'}
{'	-gamma	1e10 	0.075\n'}
{'  \n'}
{'PHASES  \n'}
{'  \n'}
{'Fix_H+\n'}
{'    H+ = H+ ; log_k 0.0\n'}
{'Fix_pe\n'}
{'    e- = e- ; log_k 0.0\n'}
{'O2(g)\n'}
{'        O2 = O2\n'}
{'        log_k           -2.960\n'}
{'  \n'}
{'H2(g) \n'}
{'        H2 = H2\n'}
{'        log_k           -3.150\n'}
];
[nolinesDATABASE,length]=size(MINIMALDATABASE);

In [3]:
fileID=fopen('MINIMALH2O.dat','a');

for i=1:nolinesDATABASE
    line=cell2mat(MINIMALDATABASE(i,:));
    fprintf(fileID,line)
end
fprintf(fileID,'\n');
fclose(fileID);

# generate the "run" file

In [4]:
RUNFILE=[...
{'SOLUTION_MASTER_SPECIES\n'}
{'Ac	    Ac-	        0.0	1		1\n'}
{' \n'}
{'SOLUTION_SPECIES\n'} 
{'#Acetic acid\n'}
{'Ac- = Ac-\n'}
{'       log_k   0.0;\n'}
{'        -gamma          1e10     0.0\n'}
{'Ac- + H+ = HAc\n'}
{'        log_k   4.75;\n'} 
{'        -gamma          1e10     0.0\n'}
{' \n'}
{'SOLUTION 1\n'}
{'    temp 25\n'}
{'    pH 4 \n'}
{'    pe 13\n'}
{'    Ac 0.01\n'}
{'-units mol/kgw\n'}
{'END\n'}
{' \n'}
{'USE solution 1 \n'}
{'EQUILIBRIUM_PHASES \n'}
{'       Fix_H+ -4          NaOH 10.0\n'}
{'-force_equality true\n'}
{'       Fix_pe -13          O2\n'}
{'-force_equality true\n'}
{' \n'}
{'SELECTED_OUTPUT\n'}
{'        -file outpunch.txt\n'}
{'        -selected_out true\n'} 
{'        -user_punch true\n'}
{'        -high_precision true\n'}
{'        -reset false\n'}
{'        -simulation false\n'}
{'        -state false\n'}
{'        -distance false\n'}
{'        -time false\n'}
{'        -step false\n'}
{'        -ph false\n'}
{'        -pe false\n'}
{'        -reaction false\n'}
{'        -temperature false\n'}
{'        -alkalinity false\n'}
{'        -ionic_strength false\n'}
{'        -water false\n'}
{'        -charge_balance false\n'}
{'        -percent_error false\n'}
{'        -molalities \n'} % list species here for outputs, but userpunch is better
{' \n'}
{'USER_PUNCH\n'}
{'-headings pH Ac HAc\n'}
{' -start \n'}
{' 10 PUNCH -LA("H+"), MOL("Ac-"), MOL("HAc")\n'}
{' -end\n'}
];
[nolinesRUN,length]=size(RUNFILE);

In [5]:
fileID=fopen('runfile.txt','a');

for i=1:nolinesRUN
    line=cell2mat(RUNFILE(i,:));
    fprintf(fileID,line)
end
fprintf(fileID,'\n');
fprintf(fileID,'END');
fclose(fileID);

In [6]:
database=['MINIMALH2O.dat']; runfile=['runfile.txt'];
str=['system("phreeqc ',runfile,' out.txt ', database,'");'];
eval(str); % output to the screen
%evalc(str); % so no screen output
fid = fopen('outpunch.txt','rt');
hdr = strtrim(regexp(fgetl(fid),'\t','split'));
%[n,m]=size(hdr)
hdr=hdr(1:3)';
mat = cell2mat(textscan(fid,repmat('%f',1,numel(hdr))));
[nsize,msize]=size(mat);
pH=mat(1); Ac=mat(2); HAc=mat(3);
pH
Ac
HAc

pH = 4.0000e+00


Ac = 1.5098e-03


HAc = 8.4902e-03


In [7]:
% solve the speciation
Ka=10^-4.75; AcT=0.01; Kw=1e-14;
H=10^-4;
pH=-log10(H)
Ac=(Ka*AcT)./(H+Ka)
HAc=AcT-Ac

pH = 4.0000e+00


Ac = 1.5098e-03


HAc = 8.4902e-03
